In [11]:
import requests
import pandas as pd
import numpy as np

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

import folium

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [12]:
# import geocoder # import geocoder

# p = 8507
# lat_lng_coords = None
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Carlton, Melbourne'.format(p))
#     lat_lng_coords = g.latlng
#     print(lat_lng_coords)

In [13]:
# just copy suburbs from <https://en.wikipedia.org/wiki/City_of_Melbourne>

suburbs_melbourne = ['Carlton', 'Carlton North', 'Docklands', 'East Melbourne', 'Flemington', 'Kensington','Melbourne CBD', 'North Melbourne', 'Port Melbourne', 'Parkville', 'Southbank', 'South Wharf', 'South Yarra', 'West Melbourne']

In [14]:
df_suburbs_melbourne=pd.DataFrame(suburbs_melbourne, columns=['address'])
# df_suburbs_melbourne.loc[(~df_suburbs_melbourne['address'].str.contains('New South Wales')),'address'] +=', New South Wales'
# df_suburbs_melbourne['address'] = df_suburbs_melbourne['address'].str.replace(', Queensland','')
# df_suburbs_melbourne=df_suburbs_melbourne.replace(regex=r'central business district', value='CBD')
df_suburbs_melbourne['address'] += ', Melbourne'
df_suburbs_melbourne[['Suburb','Region']] = df_suburbs_melbourne.address.str.split(", ", expand=True,)
# df_suburbs_melbourne.drop(df_suburbs_melbourne[df_suburbs_melbourne['Suburb'].str.contains('Council|Randwick|Woollahra')].index,inplace=True)

df_suburbs_melbourne


,address,Suburb,Region
0,"Carlton, Melbourne",Carlton,Melbourne
1,"Carlton North, Melbourne",Carlton North,Melbourne
2,"Docklands, Melbourne",Docklands,Melbourne
3,"East Melbourne, Melbourne",East Melbourne,Melbourne
4,"Flemington, Melbourne",Flemington,Melbourne
5,"Kensington, Melbourne",Kensington,Melbourne
6,"Melbourne CBD, Melbourne",Melbourne CBD,Melbourne
7,"North Melbourne, Melbourne",North Melbourne,Melbourne
8,"Port Melbourne, Melbourne",Port Melbourne,Melbourne
9,"Parkville, Melbourne",Parkville,Melbourne


In [15]:
# print and save geolocations of suburbs
# geolocation got using Nominatim is different from 

# latitudes =[]
# longitudes =[]

# for address in df_suburbs_melbourne['address']:
#     geolocator = Nominatim(user_agent="melbourne_explorer",timeout=10)
#     location = geolocator.geocode(address) 
#     print(location)
#     latitude = location.latitude if location != None else 'NF'
#     longitude = location.longitude if location != None else 'NF'
#     print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
#     latitudes.append(latitude)
#     longitudes.append(longitude)

# print(latitudes)
# print(longitudes)

latitudes = [-37.8142176, -37.8142176, -37.8175423, -37.812498, -37.7867587, -37.7939378, -37.8141823, -37.8076092, 'NF', -37.7871148, -37.8253618, -37.8253279, 'NF', -37.81044755]

longitudes = [144.9631608, 144.9631608, 144.9394923, 144.9858851, 144.9193668, 144.9305645, 144.9598007, 144.9423514, 'NF', 144.9515533, 144.9640203, 144.9493432, 'NF', 144.92042979350134]

In [16]:
df_suburbs_melbourne=df_suburbs_melbourne.drop(columns=['address'])

In [17]:
df_suburbs_melbourne['Latitude'] = latitudes
df_suburbs_melbourne['Longitude'] = longitudes
df_suburbs_melbourne['Region'] = 'City of melbourne'
df_suburbs_melbourne=df_suburbs_melbourne.reset_index(drop=True)

# find NF
df_suburbs_melbourne.loc[df_suburbs_melbourne['Latitude'] == 'NF']


,Suburb,Region,Latitude,Longitude
8,Port Melbourne,City of melbourne,NF,NF
12,South Yarra,City of melbourne,NF,NF


In [22]:
# fill NF
df_suburbs_melbourne.loc[df_suburbs_melbourne['Suburb'] == 'Port Melbourne', ['Latitude', 'Longitude']] = '-37.8332', '144.9125'
df_suburbs_melbourne.loc[df_suburbs_melbourne['Suburb'] == 'South Yarra' , ['Latitude', 'Longitude']] = '-37.8400', '144.9890'
# correct latitude and longitude
df_suburbs_melbourne.loc[df_suburbs_melbourne['Suburb'] == 'Carlton North' , ['Latitude', 'Longitude']] = '-37.7888', '144.9720'
df_suburbs_melbourne.loc[df_suburbs_melbourne['Suburb'] == 'Carlton' , ['Latitude', 'Longitude']] = '-37.8001', '144.9671'

df_suburbs_melbourne.to_csv('suburbs_melbourne.csv')
df_suburbs_melbourne

,Suburb,Region,Latitude,Longitude
0,Carlton,City of melbourne,-37.8001,144.9671
1,Carlton North,City of melbourne,-37.7888,144.9720
2,Docklands,City of melbourne,-37.8175,144.939
3,East Melbourne,City of melbourne,-37.8125,144.986
4,Flemington,City of melbourne,-37.7868,144.919
5,Kensington,City of melbourne,-37.7939,144.931
6,Melbourne CBD,City of melbourne,-37.8142,144.96
7,North Melbourne,City of melbourne,-37.8076,144.942
8,Port Melbourne,City of melbourne,-37.8332,144.9125
9,Parkville,City of melbourne,-37.7871,144.952


In [21]:
address = 'City of melbourne, Australia'

geolocator = Nominatim(user_agent="melbourne_explorer")
location = geolocator.geocode(address)
melbourne_latitude = location.latitude
melbourne_longitude = location.longitude
print('The geograpical coordinate of melbourne are {}, {}.'.format(melbourne_latitude, melbourne_longitude))

# create map of melbourne using latitude and longitude values
map_melbourne = folium.Map(location=[melbourne_latitude, melbourne_longitude], zoom_start=12)

# add markers to map
for lat, lng, region, suburb in zip(df_suburbs_melbourne['Latitude'], df_suburbs_melbourne['Longitude'], df_suburbs_melbourne['Region'], df_suburbs_melbourne['Suburb']):
    label = '{}, {}'.format(suburb, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)

map_melbourne

The geograpical coordinate of melbourne are -37.8142176, 144.9631608.
